## 3. OpenAI API Example: Ask GPT-4o a Static Question

This example shows how to use the OpenAI API to send a question to the GPT-4o model and get a response. This involves making an HTTP request to OpenAI's servers.

**Important Note**: To run this example, you need an OpenAI API key. 
1.  If you don't have one, sign up at [OpenAI's website](https://platform.openai.com/).
2.  Replace `'YOUR_API_KEY'` in the code below with your actual API key.
3.  **Keep your API key secret!** Do not share it publicly or commit it to version control.

You will also need an active internet connection.

### 3.1 Importing the OpenAI Library

-   `openai`: The official Python library for interacting with the OpenAI API.

In [1]:
# We need to install the openai library if you haven't already: !pip install openai
import openai
from openai import OpenAI # Recommended way to import client
import os # To potentially use environment variables for API key

### 3.2 Initializing the OpenAI Client

We create an instance of the `OpenAI` client, providing our API key.

**Security Best Practice**: It's better to store your API key as an environment variable rather than hardcoding it. 
For example, you could set an environment variable `OPENAI_API_KEY` and then use `api_key=os.environ.get('OPENAI_API_KEY')`.
For simplicity in this example, we'll show the direct replacement method, but **be mindful of security**.

In [2]:
# --- IMPORTANT: REPLACE 'YOUR_API_KEY' WITH YOUR ACTUAL KEY --- 
# api_key_value = "YOUR_API_KEY" 
# For safety, it's better to use an environment variable or input prompt:
try:
    # Attempt to get API key from environment variable first
    api_key_value = os.environ["OPENAI_API_KEY"]
    if not api_key_value:
        raise KeyError
    print("Using API key from environment variable OPENAI_API_KEY.")
except KeyError:
    print("OPENAI_API_KEY environment variable not found.")
    # Fallback: prompt user for API key if not in environment
    # In a real notebook, you might use input() but be careful with visibility
    api_key_value = "YOUR_API_KEY_PLACEHOLDER" # Replace this or use input()
    if api_key_value == "YOUR_API_KEY_PLACEHOLDER":
         print("Please replace 'YOUR_API_KEY_PLACEHOLDER' with your actual key or set the OPENAI_API_KEY environment variable.")
         # You might want to raise an error or skip this section if no key is provided
         # For now, we'll let it proceed, but it will fail if the key is invalid.

client = None
if api_key_value and api_key_value != "YOUR_API_KEY_PLACEHOLDER":
    try:
        client = OpenAI(api_key=api_key_value)
        print("OpenAI client initialized.")
    except Exception as e:
        print(f"Error initializing OpenAI client: {e}")
else:
    print("OpenAI client NOT initialized. Please provide a valid API key.")

Using API key from environment variable OPENAI_API_KEY.
OpenAI client initialized.


### 3.3 Defining the Question and Making the API Request

-   `question`: The question we want to ask the model.
-   `client.chat.completions.create(...)`: This is the method used to interact with chat-based models like GPT-3.5-turbo, GPT-4, and GPT-4o.
    -   `model="gpt-4o"`: Specifies which model to use. "gpt-4o" is OpenAI's latest flagship model (as of this writing).
    -   `messages=[...]`: A list of message objects that form the conversation history.
        -   `{"role": "system", "content": "You are a helpful assistant."}`: The system message sets the context or behavior for the assistant.
        -   `{"role": "user", "content": question}`: The user's message (our question).
    -   `max_tokens=50`: Limits the maximum length of the generated response. One token is roughly 4 characters of text for English.
    -   `temperature=0.7`: Controls the randomness of the output. 
        -   Higher values (e.g., 0.7-1.0) make the output more random and creative.
        -   Lower values (e.g., 0.1-0.3) make it more focused and deterministic.
        -   A value of 0 would make it mostly deterministic, but not always perfectly so.

In [3]:
if client: # Only proceed if client was initialized
    # Define a static question
    question = "What is the capital city of France?"

    try:
        # Make a request to GPT-4o
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a concise and helpful assistant."},
                {"role": "user", "content": question}
            ],
            max_tokens=50, # Maximum number of tokens in the response
            temperature=0.7  # Controls randomness: 0.0 (deterministic) to 2.0 (very random)
        )

        # Extract and print the response
        # The response structure is a choice object, we need to access the message content
        answer = response.choices[0].message.content
        print(f"\nQuestion: {question}")
        print(f"Answer: {answer.strip()}") # .strip() removes leading/trailing whitespace

    except openai.APIConnectionError as e:
        print(f"OpenAI API request failed to connect: {e}")
    except openai.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
    except openai.AuthenticationError as e:
        print(f"OpenAI API authentication failed. Check your API key: {e}")
    except openai.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("\nSkipping OpenAI API call as the client was not initialized (API key missing or invalid).")


Question: What is the capital city of France?
Answer: The capital city of France is Paris.


### 3.4 OpenAI API Practice Variation: Different Question and Parameters

Let's try asking a different question and adjusting the `max_tokens` and `temperature` parameters to see how they affect the output.

-   **New Question**: "Explain the concept of a 'tensor' in machine learning in one sentence."
-   **`max_tokens`**: We might increase this slightly if we expect a slightly longer answer, or keep it controlled for brevity.
-   **`temperature`**: Let's try a lower temperature (e.g., 0.2) for a more deterministic, factual answer, and a higher temperature (e.g., 0.9) for a more creative (though potentially less accurate for factual questions) response.

In [4]:
if client: # Only proceed if client was initialized
    new_question = "Explain the concept of a 'tensor' in machine learning in one or two simple sentences."

    print(f"\n--- OpenAI Variation 1: Different Question, Lower Temperature ---")
    try:
        response_var1 = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful and concise AI assistant, skilled at explaining complex topics simply."},
                {"role": "user", "content": new_question}
            ],
            max_tokens=80,  # Slightly more tokens allowed
            temperature=0.2   # Lower temperature for more focused response
        )
        answer_var1 = response_var1.choices[0].message.content
        print(f"Question: {new_question}")
        print(f"Answer (temp 0.2): {answer_var1.strip()}")
    except Exception as e:
        print(f"Error during OpenAI API call (Variation 1): {e}")

    print(f"\n--- OpenAI Variation 2: Same Question, Higher Temperature ---")
    try:
        response_var2 = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful and somewhat creative AI assistant."}, 
                {"role": "user", "content": new_question} 
            ],
            max_tokens=80,
            temperature=0.9   # Higher temperature for more creative/varied response
        )
        answer_var2 = response_var2.choices[0].message.content
        print(f"Question: {new_question}")
        print(f"Answer (temp 0.9): {answer_var2.strip()}")
    except Exception as e:
        print(f"Error during OpenAI API call (Variation 2): {e}")
else:
    print("\nSkipping OpenAI API variations as the client was not initialized.")


--- OpenAI Variation 1: Different Question, Lower Temperature ---
Question: Explain the concept of a 'tensor' in machine learning in one or two simple sentences.
Answer (temp 0.2): In machine learning, a tensor is a multi-dimensional array used to store data; it can be a scalar (0D), vector (1D), matrix (2D), or higher dimensions, allowing for efficient data manipulation and computation.

--- OpenAI Variation 2: Same Question, Higher Temperature ---
Question: Explain the concept of a 'tensor' in machine learning in one or two simple sentences.
Answer (temp 0.9): In machine learning, a tensor is a multi-dimensional array that generalizes scalars, vectors, and matrices. Tensors are used to represent data, inputs, and parameters in various dimensions, facilitating operations and computations in models.


By changing the `temperature`, you'll likely get different phrasings or slightly different explanations for the same question. Lower temperatures are good for factual recall, while higher temperatures can be used for more generative or brainstorming tasks (but be cautious about factual accuracy at very high temperatures).

## Conclusion and Further Learning

These examples provide a basic introduction to using PyTorch, TensorFlow, and the OpenAI API for common machine learning tasks. 

**Key Takeaways for Learning & Interviews:**

* **Understand the Core Concepts:**
    * **PyTorch**: Tensors, `nn.Module` for model building, autograd for gradients, optimizers, and the training loop structure.
    * **TensorFlow (Keras)**: The `Sequential` or Functional API for model building, layers, compilation (optimizer, loss, metrics), and the `fit`/`evaluate` workflow.
    * **OpenAI API**: Client initialization, structuring messages for chat completions, understanding parameters like `model`, `temperature`, and `max_tokens`, and how to parse the response.
* **Practice Explaining Code**: Be able to walk through each part of the code and explain *why* certain choices were made (e.g., why use `relu` vs `sigmoid`, why `binary_crossentropy` for this problem).
* **Experiment with Variations**: Actively try modifying these examples. Change architectures, try different optimizers/loss functions, explore other datasets, or ask more complex questions to the OpenAI API. This is the best way to solidify your understanding.
* **Setup and Dependencies**: Know how to install the necessary libraries (`pip install ...`) and be aware of potential environment issues (e.g., Python versions, CUDA for GPU).
* **Error Handling**: For API calls (like OpenAI), understand common errors (authentication, rate limits, connection issues) and how you might handle them in a real application.

This notebook is just a starting point. The field of ML is vast and constantly evolving. Continue exploring documentation, tutorials, and building your own projects to deepen your knowledge!